In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Flatten,Dense,Dropout,Conv2D,MaxPooling2D,Input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
base_model=VGG16(input_shape=(224,224,3),include_top=False)
base_model.trainable=False
x=Flatten()(base_model.output)
x=Dense(512,activation='relu')(x)
out=Dense(196,activation='softmax')(x)

model=Model(inputs=base_model.inputs,outputs=out)


model.summary()

print(len(model.layers))
for layer in model.layers:
  print(layer.trainable)

for layer in model.layers[:10]: ## trainable ayarlama
  layer.trainable=False

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
trainPath='/content/drive/MyDrive/Deep_Learn/Car Dataset/train'
testPath='/content/drive/MyDrive/Deep_Learn/Car Dataset/test'

trainDataGen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)
testDataGen=ImageDataGenerator(
    rescale=1./255
)
trainGen=trainDataGen.flow_from_directory(
    trainPath,
    target_size=(224,224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    subset='training'
)
valGen=trainDataGen.flow_from_directory(
    trainPath,
    target_size=(224,224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    subset='validation'
)
testGen=trainDataGen.flow_from_directory(
    testPath,
    target_size=(224,224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=1,
    
)

Found 7410 images belonging to 196 classes.
Found 734 images belonging to 196 classes.
Found 8056 images belonging to 196 classes.


In [ ]:
model.compile(loss="categorical_crossentropy",optimizer='SGD',metrics=['accuracy'])
history=model.fit(
    trainGen,
    steps_per_epoch=trainGen.samples//32,
    validation_data=valGen,
    epochs=10,
    validation_steps=valGen.samples//32
)

Epoch 1/10
 34/231 [===>..........................] - ETA: 24:13 - loss: 5.4258 - accuracy: 0.0074

KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

callbacks=[EarlyStopping(monitor='val_loss',patience=5),ModelCheckpoint(filepath="best_model.h5",monitor='val_loss',save_best_only=True)]
history=model.fit(
    trainGen,
    steps_per_epoch=trainGen.samples//32,
    validation_data=valGen,
    epochs=10,
    validation_steps=valGen.samples//32
    callbacks=callbacks
)